# Generation of PMO Version 2.0
## 28 May 2024

In [1]:
import datetime
import pathlib
import urllib.parse
import xml.etree.ElementTree as etree

import rdflib

from jinja2 import Template

DC = rdflib.Namespace("http://purl.org/dc/terms/")
OWL = rdflib.Namespace("http://www.w3.org/2002/07/owl#")
RDACT = rdflib.Namespace("http://rdaregistry.info/termList/RDACarrierType/")
SKOS = rdflib.Namespace("http://www.w3.org/2004/02/skos/core#")

In [2]:
rdf_template = """<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF xmlns:dcterms="http://purl.org/dc/terms/"
         xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
         xmlns:skos="http://www.w3.org/2004/02/skos/core#"
         xmlns:rdact="http://rdaregistry.info/termList/RDACarrierType/">
</rdf:RDF>"""

ontology_html_template ="""<!doctype html>
<html lang="en">
<head>
  <link href="/css/owl.css" rel="stylesheet" type="text/css" />
  <link href="/css/Primer.css" rel="stylesheet" type="text/css" />
  <link href="/css/rec.css" rel="stylesheet" type="text/css" />
  <link href="/css/extra.css" rel="stylesheet" type="text/css" />
  <link rel="shortcut icon" href="/css/favicon.ico" />
</head>
<body>
    <h1>{label}</h1>
    <p>
      Documentation at <a href="/2.0/ontologies/vocabularies/{html_doc}">{label}</a>.
    </p>
    <ul>
    {entities_list}
    </ul>
</body>
</html>"""

html_template = Template("""<!doctype html>
<html lang="en">
<head>
  <link href="/css/owl.css" rel="stylesheet" type="text/css" />
  <link href="/css/Primer.css" rel="stylesheet" type="text/css" />
  <link href="/css/rec.css" rel="stylesheet" type="text/css" />
  <link href="/css/extra.css" rel="stylesheet" type="text/css" />
  <link rel="shortcut icon" href="/css/favicon.ico" />
</head>
<body>
  <h1>{{label}}</h1>
  <p>
      Documentation at <a href='/2.0/vocabularies/{{html_doc}}.html#{{name}}'>{{label}}</a>.
      Other versions:
      <ul>
          <li><a href="/2.0/vocabularies/{{ontology_name}}/{{name}}/{{name}}.rdf">RDF XML</a></li>
          <li><a href="/2.0/vocabularies/{{ontology_name}}/{{name}}/{{name}}.ttl">Turtle</a></li>
      </ul>
  </p>
</body>
</html>""")

In [3]:
main_html_template = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<link href="/css/owl.css" rel="stylesheet" type="text/css">
<link href="/css/Primer.css" rel="stylesheet" type="text/css">
<link href="/css/rec.css" rel="stylesheet" type="text/css">
<link href="/css/extra.css" rel="stylesheet" type="text/css">
<link rel="shortcut icon" href="/PerformedMusicOntology/css/favicon.ico">
<script src="/js/jquery.js"></script>
<script src="/PerformedMusicOntology/js/jquery.scrollTo.js"></script>
<script src="/PerformedMusicOntology/js/marked.min.js"></script>
<title>{{ vocab.title }}</title>
</head>
<body>
<div class="head">
<h3>{{ vocab.title }}</h3>
<p>{{ vocab.url }}</p>
<div class="comment">
  <span class="markdown">
    <p>{{ vocab.definition }}</p>
  </span>
  <p><a href="{{ vocab.url }}.rdf">Vocabulary source</a></p>
</div>
<hr>
<div id="toc">
  <h2>Table of Contents</h2>
  <ol>
   <li><a href="#classes">Classes</a></li>
   <li><a href="#namespacedeclarations">Namespace Declarations</a></li>
  </ol>
</div>
<div id="classes">
  <h2>Classes</h2>
  <ul class="hlist">
  {% for class_ in classes %}
    <li>
      <a href="#{{ class_.id }}" title="{{ class_.url }}">
        <span>{{ class_.prefLabel }}</span>
      </a>
    </li>
  {% endfor %}
  </ul>
</div>
</div>
{% for class_ in classes %}
<div id="{{ class_.id }}" class="entity">
  <a name="{{ class_.url }}"></a>
  <h3>
   {{ class_.prefLabel }}<sup title="class" class="type-c">c</sup>
   <span class="backlink"> back to <a href="#toc">ToC</a> or <a href="#classes">Class ToC</a></span>
  </h3>
  <p><strong>IRI: </strong>{{ class_.url }}</p>
  {% if class_.altLabel %}
  <p><strong>Use for: </strong>{{ class_.altLabel }}</p>
  {% endif %}
  <p><strong>Scope note: </strong>{{ class_.definition }}</p>
  <p><strong>Current version: </strong>{{ class_.changeNote }}</p>
</div>
</div>
{% endfor %}
<div id="namespacedeclarations">
  <h2>Namespace Declarations <span class="backlink"> back to <a href="#toc">ToC</a></span></h2>
    <dl>
    <dt>dcterms</dt>
    <dd>http://purl.org/dc/terms/</dd>
    <dt>rdf</dt>
    <dd>http://www.w3.org/1999/02/22-rdf-syntax-ns#</dd>
    <dt>rdfs</dt>
    <dd>http://www.w3.org/2000/01/rdf-schema#</dd>
    <dt>skos</dt>
    <dd>http://www.w3.org/2004/02/skos/core#</dd>
    </dl>
</div>
</body>
</html>
""")

In [4]:
version_2_0 = pathlib.Path("/Users/jpnelson/30-39 LD4P, PCC, Sinopia, and FOLIO/30.09 LD4P Performed Music Ontology/2.0")

In [5]:
def init_ontology(**kwargs):
    rdf_path = pathlib.Path(kwargs.get("rdf_path"))
    html_doc_file = kwargs.get("html_doc_file")
    start = datetime.datetime.now(datetime.UTC)
    print("{} Staring ontology initialization".format(start.isoformat()))
    if not rdf_path.exists():
        raise ValueError(f"{rdf_path} rdf_path doesn't exist")
    ontology_graph = rdflib.Graph()
    ontology_graph.parse(rdf_path, format='turtle')
    ontology_entity = ontology_graph.value(predicate=rdflib.RDF.type, object=SKOS.ConceptScheme)
    ontology_label = ontology_graph.value(subject=ontology_entity, predicate=rdflib.RDFS.label)
    entities = list(ontology_graph.subjects(predicate=rdflib.RDF.type, object=SKOS.Concept))
    ontology_uri = urllib.parse.urlparse(str(ontology_entity))
    ontology_path = pathlib.Path(f"../{ontology_uri.path}")
    print(ontology_path.absolute(), ontology_path.exists())
    if not ontology_path.exists():
        ontology_path.mkdir(parents=True, exist_ok=True)
    generate_vocab_home(
        ontology_entity,
        ontology_graph,
        ontology_path,
        html_doc_file,
        entities)
    entity_list_html = ''
    for i,row in enumerate(entities):
        if i > 0 and not i%10:
            print(f"{i:2}", end="")
        else:
            print(".", end="")
        entity_list_html += "{}\n".format(
            init_entity(entity=row,
                        ontology_name=ontology_label,
                        ontology_path=ontology_path,
                        ontology_graph=ontology_graph,
                        html_doc_file=html_doc_file)
            )
    raw_html = ontology_html_template.format(**{
            "label": ontology_label,
            "html_doc": html_doc_file,
            "entities_list": entity_list_html})
    with (ontology_path / "index.html").open("w+") as fo:
        fo.write(raw_html)
    end = datetime.datetime.now(datetime.UTC)
    print(f"\n{end.isoformat()} - Finished {ontology_label}, total time {(end-start).seconds / 60.0:,} minutes")

def generate_vocab_home(ontology_entity, graph, ontology_path, vocab_name, entities):
    vocab = {
        "title": graph.value(subject=ontology_entity, predicate=DC.title),
        "url": str(ontology_entity),
        "definition":  graph.value(subject=ontology_entity, predicate=SKOS.definition)
    }
    classes = []
    for entity in entities:
        class_ = {
            "id": str(entity).split("/")[-1],
            "definition": graph.value(subject=entity, predicate=SKOS.definition),
            "prefLabel":  graph.value(subject=entity, predicate=SKOS.prefLabel),
            "changeNote": _change_note(entity, graph),
            "url": str(entity)
        
        }
        alt_label = graph.value(subject=entity, predicate=SKOS.altLabel)
        if alt_label:
            class_["altLabel"] = alt_label
        classes.append(class_)
    vocab_home_path = ontology_path.parent / f"{vocab_name}.html"
    with vocab_home_path.open("w+") as fo:
        fo.write(main_html_template.render(vocab=vocab, classes=classes))
    rdf_vocab_home_path = ontology_path.parent / f"{vocab_name}.rdf"
    with rdf_vocab_home_path.open("w+") as fo:
        fo.write(graph.serialize(format='xml'))
    ttl_vocab_home_path = ontology_path.parent / f"{vocab_name}.ttl"
    with ttl_vocab_home_path.open("w+") as fo:
        fo.write(graph.serialize(format='turtle'))
                 
def _change_note(uri, graph):
    change_note_bnode = graph.value(subject=uri, predicate=SKOS.changeNote)
    note_val = graph.value(subject=change_note_bnode, predicate=rdflib.RDF.value)
    date = graph.value(subject=change_note_bnode, predicate=DC.date)
    return f"{note_val} {date}"

def init_entity(**kwargs):
    entity = kwargs.get('entity')
    ontology_name = kwargs.get("ontology_name")
    ontology_path = kwargs.get("ontology_path")
    ontology_graph = kwargs.get("ontology_graph")
    html_doc_file = kwargs.get("html_doc_file")

    name = str(entity).split("/")[-1]

    # Make sub-directory if it doesn't exist
    entity_path = ontology_path / name
    if not entity_path.exists():
        entity_path.mkdir(parents=True, exist_ok=True)
    # Create an HTML file that redirects to Entity
    html_path = entity_path / "index.html"
    label = ontology_graph.value(subject=entity, predicate=SKOS.prefLabel)
  
    with html_path.open("w+") as fo:
        fo.write(html_template.render(html_doc=html_doc_file,
                                      ontology_name=ontology_path.name,
                                      label=label, 
                                      name=name))

    graph = rdflib.ConjunctiveGraph()
    graph.namespace_manager.bind("dc", DC)
    graph.namespace_manager.bind("rdact", RDACT)
    graph.namespace_manager.bind("skos", SKOS)
    for pred, obj in ontology_graph.predicate_objects(entity):
        graph.add((entity, pred, obj))
        if isinstance(obj, rdflib.BNode):
            for bnode_pred, bnode_obj in ontology_graph.predicate_objects(obj):
                graph.add((obj, bnode_pred, bnode_obj))
    # Writes an RDF XML file
    rdf_xml_path = entity_path / f"{name}.rdf"
    save_rdf(graph, rdf_xml_path, "xml")
    # Writes a RDF Turtle file
    rdf_ttl_path = entity_path / f"{name}.ttl"
    save_rdf(graph, rdf_ttl_path, "ttl") 
    print("\t{time} Finished initializing {label}".format(
          **{ "time": datetime.datetime.now(datetime.UTC).isoformat(),
            "label": label }))
    
    return f"""<li><a href="{entity}">{label}</li>"""
    
def save_rdf(graph, path, serialize_as):
    print("\t\t{} Saving {}, {} triples as {}".format(datetime.datetime.now(datetime.UTC).isoformat(),
                                              path,
                                              len(graph),
                                              serialize_as))
    #path.mkdir(parents=True, exist_ok=True)
    with path.open("w+") as fo:
        fo.write(graph.serialize(format=serialize_as))
    print(".", end="")

def generate_resource(subject: rdflib.URIRef):
    name = str(subject).split("/")[-1]
    resource_graph = rdflib.ConjunctiveGraph()
    resource_graph.namespace_manager.bind("dc", DC)
    resource_graph.namespace_manager.bind("rdact", RDACT)
    resource_graph.namespace_manager.bind("skos", SKOS)
    resource_path = ontology_path / name
    resource_path.mkdir(exist_ok=True)
    # label = pmo_2_0.value(subject=subject, predicate=rdflib.RDFS.label)
    # if label is None:
    #     label = name
    # definition = pmo_2_0.value(subject=subject, predicate=SKOS.definition)
    # raw_html = html_template.format(name=name, label=label, definition=definition)
    # index_path = resource_path/"index.html"
    # index_path.write_text(raw_html)
    # Populate resource graph
    # for pred, obj in pmo_2_0.predicate_objects(subject=subject):
    #     resource_graph.add((subject, pred, obj))
    # Write RDF XML and Turtle serialization
    # rdf_path = resource_path/f"{name}.rdf"
    # rdf_path.write_text(resource_graph.serialize(format='xml'))
    # ttl_path = resource_path/f"{name}.ttl"
    # ttl_path.write_text(resource_graph.serialize(format='turtle'))

In [6]:
version_2_0.absolute()

PosixPath('/Users/jpnelson/30-39 LD4P, PCC, Sinopia, and FOLIO/30.09 LD4P Performed Music Ontology/2.0')

## Ensemble Size

In [7]:
init_ontology(rdf_path = (version_2_0 / "Ontology/Vocabularies/EnsembleSize/Ensemble_size.ttl"),
              html_doc_file="PMOEnsembleSize")

2024-05-28T21:13:32.754903+00:00 Staring ontology initialization
/Users/jpnelson/30-39 LD4P, PCC, Sinopia, and FOLIO/30.09 LD4P Performed Music Ontology/Documentation/../2.0/vocabularies/ensemble_size False
.		2024-05-28T21:13:32.762247+00:00 Saving ../2.0/vocabularies/ensemble_size/solo_or_individual/solo_or_individual.rdf, 8 triples as xml
.		2024-05-28T21:13:32.762438+00:00 Saving ../2.0/vocabularies/ensemble_size/solo_or_individual/solo_or_individual.ttl, 8 triples as ttl
.	2024-05-28T21:13:32.762658+00:00 Finished initializing solo or individual
.		2024-05-28T21:13:32.762944+00:00 Saving ../2.0/vocabularies/ensemble_size/duo/duo.rdf, 9 triples as xml
.		2024-05-28T21:13:32.763116+00:00 Saving ../2.0/vocabularies/ensemble_size/duo/duo.ttl, 9 triples as ttl
.	2024-05-28T21:13:32.763318+00:00 Finished initializing duo
.		2024-05-28T21:13:32.763592+00:00 Saving ../2.0/vocabularies/ensemble_size/trio/trio.rdf, 8 triples as xml
.		2024-05-28T21:13:32.763756+00:00 Saving ../2.0/vocabular

## EventType

In [8]:
init_ontology(rdf_path = (version_2_0 / "Ontology/Vocabularies/EventType/Event_type.ttl"),
              html_doc_file="PMOEventType")

2024-05-28T21:13:32.772059+00:00 Staring ontology initialization
/Users/jpnelson/30-39 LD4P, PCC, Sinopia, and FOLIO/30.09 LD4P Performed Music Ontology/Documentation/../2.0/vocabularies/event_type False
.		2024-05-28T21:13:32.777957+00:00 Saving ../2.0/vocabularies/event_type/audition/audition.rdf, 8 triples as xml
.		2024-05-28T21:13:32.778177+00:00 Saving ../2.0/vocabularies/event_type/audition/audition.ttl, 8 triples as ttl
.	2024-05-28T21:13:32.778396+00:00 Finished initializing Audition
.		2024-05-28T21:13:32.778676+00:00 Saving ../2.0/vocabularies/event_type/benefit_concert/benefit_concert.rdf, 8 triples as xml
.		2024-05-28T21:13:32.778837+00:00 Saving ../2.0/vocabularies/event_type/benefit_concert/benefit_concert.ttl, 8 triples as ttl
.	2024-05-28T21:13:32.779032+00:00 Finished initializing Benefit concert
.		2024-05-28T21:13:32.779295+00:00 Saving ../2.0/vocabularies/event_type/ceremony/ceremony.rdf, 8 triples as xml
.		2024-05-28T21:13:32.779459+00:00 Saving ../2.0/vocabular

## Medium Component Qualifier

In [9]:
init_ontology(rdf_path = (version_2_0 / "Ontology/Vocabularies/MediumComponentQualifier/Medium_component_qualifier.ttl"),
              html_doc_file="PMOMediumComponentQualifier")

2024-05-28T21:13:32.788462+00:00 Staring ontology initialization
/Users/jpnelson/30-39 LD4P, PCC, Sinopia, and FOLIO/30.09 LD4P Performed Music Ontology/Documentation/../2.0/vocabularies/medium_component_qualifier False
.		2024-05-28T21:13:32.792123+00:00 Saving ../2.0/vocabularies/medium_component_qualifier/ad_lib/ad_lib.rdf, 9 triples as xml
.		2024-05-28T21:13:32.792287+00:00 Saving ../2.0/vocabularies/medium_component_qualifier/ad_lib/ad_lib.ttl, 9 triples as ttl
.	2024-05-28T21:13:32.792487+00:00 Finished initializing ad lib
.		2024-05-28T21:13:32.792767+00:00 Saving ../2.0/vocabularies/medium_component_qualifier/amplified/amplified.rdf, 8 triples as xml
.		2024-05-28T21:13:32.792942+00:00 Saving ../2.0/vocabularies/medium_component_qualifier/amplified/amplified.ttl, 8 triples as ttl
.	2024-05-28T21:13:32.793150+00:00 Finished initializing amplified
.		2024-05-28T21:13:32.793441+00:00 Saving ../2.0/vocabularies/medium_component_qualifier/obbligato/obbligato.rdf, 8 triples as xml
.

In [10]:
def convert_ttl_rdf_xml(rdf_filepath, output_path):
    graph = rdflib.Graph()
    graph.parse(rdf_filepath, format='turtle')
    with output_path.open("w+") as fo:
        fo.write(graph.serialize(format='xml'))


In [11]:
convert_ttl_rdf_xml(
    (version_2_0 / "Ontology/Vocabularies/EnsembleSize/Ensemble_size.ttl"),
    (version_2_0 / "vocabularies/ensemble_size/PMOEnsembleSize.rdf"))

In [12]:
convert_ttl_rdf_xml(
    (version_2_0 / "Ontology/Vocabularies/EventType/Event_type.ttl"),
    (version_2_0 / "vocabularies/event_type/PMOEventType.rdf"))

In [13]:
convert_ttl_rdf_xml(
    (version_2_0 / "Ontology/Vocabularies/MediumComponentQualifier/Medium_component_qualifier.ttl"),
    (version_2_0 / "vocabularies/medium_component_qualifier/PMOMediumComponentQualifier.rdf"))

## PMO Version 2.0 Page

In [14]:
template = Template("""<?xml version="1.0" encoding="UTF-8"?>
<html xmlns="http://www.w3.org/1999/xhtml">
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <link href="/css/owl.css" rel="stylesheet" type="text/css" />
    <link href="/css/Primer.css" rel="stylesheet" type="text/css" />
    <link href="/css/rec.css" rel="stylesheet" type="text/css" />
    <link href="/css/extra.css" rel="stylesheet" type="text/css" />
    <link rel="shortcut icon" href="/css/favicon.ico" />
    <script src="/js/jquery.js"></script>
    <script src="/js/jquery.scrollTo.js"></script>
    <script src="/PerformedMusicOntology/js/marked.min.js"></script>
    <script>
      $(document).ready(
            function () {
                jQuery(".markdown").each(function(el){
                    jQuery(this).after(marked(jQuery(this).text())).remove()});
                var list = $('a[name="http://www.essepuntato.it/static/tmp/0"]');
                if (list.size() != 0) {
                    var element = list.first();
                    $.scrollTo(element);
                }
        });
    </script>
  </head>
  <body>
    <div class="head">
      <h1>Performed Music Ontology 2.0</h1>
      <dl>
        <dt>IRI:</dt>
        <dd>http://performedmusicontology.org/2.0/ontology/</dd>
        <dt>Version IRI:</dt>
        <dd>http://performedmusicontology.org/2.0/ontology/</dd>
       </dl>
       <dl>
         <dt>Other visualisation:</dt>
         <dd><a href="http://performedmusicontology.org/2.0/ontology/PerformedMusicOntology.rdf">Ontology source</a></dd>
        </dl>
     </div>
     <hr />
     <div id="toc">
       <h2>Table of Content</h2>
       <ol>
          <li><a href="#classes">Classes</a></li>
          <li><a href="#objectproperties">Object Properties</a></li>
          <li><a href="#dataproperties">Data Properties</a></li>
          <li><a href="#namespacedeclarations">Namespace Declarations</a></li>
        </ol>
     </div>
     <div id="classes">
        <h2>Classes</h2>
        <ul class="hlist">
          {% for class_ in classes %}
            <li>
              <a href="#{{ class_.id }}" title="{{ class_.uri }}">
                <span>{{ class_.label }}</span>
              </a>
            </li>
          {% endfor %}
        </ul>
        {% for class_ in classes %}
        <div id="{{ class_.id }}" class="entity">
          <a name="{{ class_.uri }}"></a>
          <h3>
           {{ class_.label }}<sup title="class" class="type-c">c</sup>
           <span class="backlink"> back to <a href="#toc">ToC</a> or <a href="#classes">Class ToC</a></span>
          </h3>
          <p><strong>IRI: </strong>{{ class_.uri }}</p>
          <div class="comment">
           {% if class_.definition %}
            <span class="markdown">{{ class_.definition }}</span>
           {% endif %}
           {% if class_.example %}
            {% for example in class_.example %}
            <span class="markdown">{{ example }}</span>
            {% endfor %}
           {% endif %}
          </div>
          {% if class_.superClass or class_.children %}
           <dl class="description">
           {% if class_.superClass %}
             <dt>has super-classes</dt>
             {% for row in class_.superClass %}
             <dd>
               <span class="dotted" title="{{ row.uri }}">{{ row.label }}</span>
             </dd>
             {% endfor %}
           {% endif %}
           {% if class_.children %}
            <dt>has sub-classes</dt>
            {% for child in class_.children %}
            <dd>
               <a href="{{ child.uri }}">{{ child.label }}</a>
               <sup title="class" class="type-c">c</sup>{% if not loop.last %},{% endif %}
            </dd>
            {% endfor %}
           {% endif %}
           </dl>
          {% endif %}
        </div>
        {% endfor %}
        <div id="objectproperties">
          <h2>Object Properties</h2>
          <ul class="hlist">
           {% for obj_prop in object_properties %}
            <li>
               <a href="#{{ obj_prop.id }}" title="{{ obj_prop.uri }}">
                 <span>{{ obj_prop.label }}</span>
               </a>
            </li>
           {% endfor %}
          </ul>
         {% for obj_prop in object_properties %}
          <div id="{{ obj_prop.id }}" class="entity">
            <a name="{{ obj_prop.uri }}"></a>
            <h3>
            {{ obj_prop.label }} <sup title="object property" class="type-op">op</sup>
             <span class="backlink"> back to <a href="#toc">ToC</a> or <a href="#objectproperties">Object Property ToC</a></span>
            </h3>
            <p>
               <strong>IRI:</strong>
               {{ obj_prop.uri }}
            </p>
            <div class="comment">
               {% if obj_prop.definition %}
                {% for def in obj_prop.definition %}
                 <span class="markdown">{{ def }}</span>
                {% endfor %}
               {% endif %}
               {% if obj_prop.example %}
                {% for example in obj_prop.example %}
                 <span class="markdown">{{ example }}</span>
                {% endfor %}
               {% endif %}
            </div>
            {% if obj_prop.has_description %}
            <div class="description">
              <dl>
              {% if obj_prop.super_properties|length > 0 %}
               <dt>has super-properties</dt>
               {% for row in obj_prop.super_properties %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>
               {% endfor %}
              {% endif %}
              {% if obj_prop.domain|length > 0 %}
                <dt>has domain</dt>
                {% for row in obj_prop.domain %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              {% if obj_prop.range|length > 0 %}
                <dt>has range</dt>
                {% for row in obj_prop.range %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              {% if obj_prop.inverse_of|length > 0 %}
                <dt>is inverse of</dt>
                {% for row in obj_prop.inverse_of %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              </dl>
            </div>
            {% endif %}
          </div>
         {% endfor %}
         <div id="dataproperties">
           <h2>Data Properties</h2>
           <ul class="hlist">
           {% for data_type in data_types %}
            <li>
              <a href="#{{ data_type.id }}" title="{{ data_type.uri }}">{{ data_type.label }}</a>
            </li>
           {% endfor %}
           </ul>
           {% for data_type in data_types %}
           <div id="{{ data_type.id }}" class="entity">
            <a name="{{ data_type.uri }}"></a>
            <h3>
            {{ data_type.label }}
            <sup title="data property" class="type-dp">dp</sup>
            <span class="backlink"> back to <a href="#toc">ToC</a> or <a href="#dataproperties">Data Property ToC</a></span>
            </h3>
            <p>
              <strong>IRI:</strong>
               {{ data_type.uri }}
            </p>        
             {% if data_type.definition %}
               <div class="comment">
               {% for def in data_type.definition %}
                 <p><span class="markdown">{{ def }}</span></p>
               {% endfor %}
               </div>
             {% endif %}
             {% if data_type.comments %}
                <div class="comment">
                {% for comment in data_type.comments %}
                 <p><span class="markdown">{{ comment }}</span></p>
                {% endfor %}
                </div>
             {% endif %}
             {% if data_type.has_description %}
              <div class="description">
             {% if data_type.super_properties|length > 0 %}
               <dt>has super-properties</dt>
               {% for row in data_type.super_properties %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>
               {% endfor %}
              {% endif %}
              {% if data_type.domain|length > 0 %}
                <dt>has domain</dt>
                {% for row in data_type.domain %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              {% if data_type.range|length > 0 %}
                <dt>has range</dt>
                {% for row in data_type.range %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              {% if data_type.inverse_of|length > 0 %}
                <dt>is inverse of</dt>
                {% for row in data_type.inverse_of %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              </div>
             {% endif %}
           </div>
           {% endfor %}
         </div>
        </div>
      </div>
      <div id="namespacedeclarations">
        <h2>Namespace Declarations <span class="backlink"> back to <a href="#toc">ToC</a></span></h2>
        <dl>
          <dt><em>default namespace</em></dt>
            <dd>http://performedmusicontology.org/ontology/</dd>
          <dt>bibframe</dt>
            <dd>http://id.loc.gov/ontologies/bibframe/</dd>
          <dt>dc</dt>
            <dd>http://purl.org/dc/elements/1.1/</dd>
          <dt>efrbroo</dt>
            <dd>http://erlangen-crm.org/efrbroo/</dd>
          <dt>owl</dt>
            <dd>http://www.w3.org/2002/07/owl#</dd>
          <dt>rdf</dt>
            <dd>http://www.w3.org/1999/02/22-rdf-syntax-ns#</dd>
          <dt>rdfs</dt>
            <dd>http://www.w3.org/2000/01/rdf-schema#</dd>
          <dt>regap</dt>
            <dd>http://metadataregistry.org/uri/profile/regap/</dd>
          <dt>skos</dt>
            <dd>http://www.w3.org/2004/02/skos/core#</dd>
          <dt>xsd</dt>
            <dd>http://www.w3.org/2001/XMLSchema#</dd>
        </dl>
    </div>
   </body>
</html>""")

In [15]:
pmo_graph = rdflib.Graph()

In [16]:
version_2_0 = pathlib.Path("/Users/jpnelson/30-39 LD4P, PCC, Sinopia, and FOLIO/30.09 LD4P Performed Music Ontology/2.0")

In [17]:
pmo_graph.parse(version_2_0 / "Ontology/PMO.rdf")

<Graph identifier=N2232ce8d6fcb490dac1b8d9e7419008d (<class 'rdflib.graph.Graph'>)>

In [18]:
object_properties = [s for s in pmo_graph.subjects(predicate=rdflib.RDF.type, object=OWL.ObjectProperty)]
print(len(object_properties))

25


In [19]:
lookups = {
    "http://id.loc.gov/ontologies/bibframe/Contribution": "contribution",
    "http://id.loc.gov/ontologies/bibframe/derivativeOf": "derivative of",
    "http://id.loc.gov/ontologies/bibframe/Event": "event",
    "http://id.loc.gov/ontologies/bibframe/eventContent": "event content",
    "http://id.loc.gov/ontologies/bibframe/eventContentOf": "event content of",
    "http://id.loc.gov/ontologies/bibframe/hasDerivative": "has derivative",
    "http://id.loc.gov/ontologies/bibframe/Identifier": "identifier",
    "http://id.loc.gov/ontologies/bibframe/Organization": "organization",
    "http://id.loc.gov/ontologies/bibframe/referencedBy": "referenced by",
    "http://id.loc.gov/ontologies/bibframe/references": "references",
    "http://id.loc.gov/ontologies/bibframe/relatedTo": "related to",
    "http://id.loc.gov/ontologies/bibframe/Work": "work"
}

In [20]:
def check_for_label(object_, graph):
    label = None
    if str(object_) in lookups:
        label = lookups[str(object_)]
    if not label:
        label = graph.value(subject=object_, predicate=rdflib.RDFS.label)
        if label:
            label = str(label)
    if label and str(object_).startswith("http://performedmusicontology.org"):
        label = create_link_super(object_, label)
    return label
        
def create_link_super(object_, label):
    ident = str(object_).split("/")[-1]
    return f"""<a href="#{ident}" title="{object_}">{label}</a>
    <sup title="object property" class="type-op">op</sup>"""

def generate_description(subject, graph, all_obj_props):
    object_prop_description = { 
        "super_properties": [],
        "domain": [],
        "range": [],
        "inverse_of": [],
        "has_description": False,
    }
    for obj in graph.objects(subject=subject, predicate=rdflib.RDFS.subPropertyOf):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject}'s {obj} super properties not found in lookups")
        object_prop_description["has_description"] = True
        object_prop_description["super_properties"].append(
            {
                "uri": str(obj),
                "label": label
            }
        )
    for obj in graph.objects(subject=subject, predicate=rdflib.RDFS.domain):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject}'s domain of {obj} not found")
        object_prop_description["has_description"] = True
        object_prop_description["domain"].append(
            {
                "uri": str(obj),
                "label": label
            }
        )
    for obj in graph.objects(subject=subject, predicate=rdflib.RDFS.range):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject}'s range of {obj} not found")
        object_prop_description["has_description"] = True
        object_prop_description["range"].append(
            {
                "uri": str(obj),
                "label": label,
            }
        )
    for obj in graph.objects(subject=subject, predicate=OWL.inverseOf):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject} inverseOf {obj} not found")
        object_prop_description["has_description"] = True
        object_prop_description["inverse_of"].append(
            {
                "uri": str(obj),
                "label": label
            }
        )
    return object_prop_description

In [21]:
classes = {}
for subj in pmo_graph.subjects(predicate=rdflib.RDF.type, object=OWL.Class):
    if str(subj) in classes:
        print(f"{subj} already exists, please fix")
        continue
    class_info = {
        "id": str(subj).split("/")[-1],
        "uri": str(subj),
        "children": []
    }
    label = pmo_graph.value(subject=subj, predicate=rdflib.RDFS.label)
    if label:
        class_info["label"] = str(label)
    else:
        class_info["label"] = class_info["id"]
    definition = pmo_graph.value(subject=subj, predicate=SKOS.definition)
    if definition:
        class_info["definition"] = str(definition)
    examples = pmo_graph.objects(subject=subj, predicate=SKOS.example)
    if examples:
        class_info["example"] = [str(i) for i in examples]
    sub_class_of = pmo_graph.objects(subject=subj, predicate=rdflib.RDFS.subClassOf)
    if sub_class_of:
        super_classes = []
        for super_class in sub_class_of:
            label = check_for_label(super_class, pmo_graph)
            if not label:
                raise ValueError(f"{subj} super-class {super_class} doesn't have a label")
            output = {
                "uri": str(super_class),
                "label": label
            }
            super_classes.append(output)
            if not str(super_class) in classes:
                classes[str(super_class)] = {"children": []}
            if not output in classes[str(super_class)]["children"]:
                classes[str(super_class)]["children"].append(output)     
        class_info["superClass"] = super_classes
    classes[class_info['uri']] = class_info

In [22]:
objects_props = {}
for subj in object_properties:
    object_property = {
      "id": str(subj).split("/")[-1],
      "uri": str(subj)
    }
    object_property.update(generate_description(subj, pmo_graph, objects_props))
    objects_props[str(subj)] = object_property
    label = pmo_graph.value(subject=subj, predicate=rdflib.RDFS.label)
    if not label:
        object_property["label"] = str(subj).split("/")[-1]
    else:
        object_property["label"] = str(label)
    definition = [str(s) for s in pmo_graph.objects(subject=subj, predicate=SKOS.definition)]
    if len(definition) > 0:
        object_property["definition"] = definition

In [23]:
objects_props['http://performedmusicontology.org/2.0/ontology/inspirationFor']

{'id': 'inspirationFor',
 'uri': 'http://performedmusicontology.org/2.0/ontology/inspirationFor',
 'super_properties': [{'uri': 'http://id.loc.gov/ontologies/bibframe/relatedTo',
   'label': 'related to'}],
 'domain': [{'uri': 'http://id.loc.gov/ontologies/bibframe/Event',
   'label': 'event'}],
 'range': [{'uri': 'http://id.loc.gov/ontologies/bibframe/Work',
   'label': 'work'}],
 'inverse_of': [],
 'has_description': True,
 'label': 'Inspiration for',
 'definition': ['Relates an event to a work that is inspired by an event.']}

## Data Types

In [24]:
data_types = {}
for subject in pmo_graph.subjects(predicate=rdflib.RDF.type, object=OWL.DatatypeProperty):
    subject_uri = str(subject)
    ident = subject_uri.split("/")[-1]
    data_type = {
        "id": ident,
        "uri": subject_uri
    }
    label = pmo_graph.value(subject=subject, predicate=rdflib.RDFS.label)
    if label:
        data_type["label"] = str(label)
    else:
        data_type["label"] = ident
    data_type.update(generate_description(subj, pmo_graph, data_types))
    definition = pmo_graph.objects(subject=subject, predicate=SKOS.definition)
    if definition:
        data_type["definition"] = [str(s) for s in definition]
    comments = pmo_graph.objects(subject=subject, predicate=rdflib.RDFS.comment)
    if comments:
        data_type["comments"] = [str(s) for s in comments]
    data_types[subject_uri] = data_type

In [25]:
data_types

{'http://performedmusicontology.org/2.0/ontology/hasMediumCount': {'id': 'hasMediumCount',
  'uri': 'http://performedmusicontology.org/2.0/ontology/hasMediumCount',
  'label': 'Has medium count',
  'super_properties': [],
  'domain': [{'uri': 'http://id.loc.gov/ontologies/bibframe/Contribution',
    'label': 'contribution'}],
  'range': [{'uri': 'http://performedmusicontology.org/2.0/ontology/MediumOfPerformance',
    'label': '<a href="#MediumOfPerformance" title="http://performedmusicontology.org/2.0/ontology/MediumOfPerformance">Medium of performance</a>\n    <sup title="object property" class="type-op">op</sup>'}],
  'inverse_of': [],
  'has_description': True,
  'definition': ['Relates a medium component to the count of that medium component.'],
  'comments': []},
 'http://performedmusicontology.org/2.0/ontology/hasNumberOfHands': {'id': 'hasNumberOfHands',
  'uri': 'http://performedmusicontology.org/2.0/ontology/hasNumberOfHands',
  'label': 'Number of hands',
  'super_properties

In [26]:
rendered_pmo = template.render(classes = classes.values(), 
                               object_properties=objects_props.values(),
                               data_types=data_types.values())
with open("/Users/jpnelson/30-39 LD4P, PCC, Sinopia, and FOLIO/30.09 LD4P Performed Music Ontology/2.0/Ontology/index.html", "w+") as fo:
    fo.write(rendered_pmo)